In [ ]:
import datetime as dt
from tinydb import TinyDB
import os
import sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path:
    sys.path.append(dir1)


In [ ]:
from services.crypto_get_data import getData
from strategys.SmaCross import SmaCrossSignal
from strategys.HiLo import HiLoSignal
from services.condicao import funcaoDeCondicao

In [ ]:
cwd = os.getcwd()
path = os.path.join("..", 'DB', 'estrategys.json')
db = TinyDB(path)
table = db.table('estrategys')

In [ ]:
def funcaoLimitadoraGerarCarteira(estrategia):
    result = estrategia['results']
    return funcaoDeCondicao(result)

In [ ]:
estrategias = table.search(funcaoLimitadoraGerarCarteira)
estrategias.sort(key=lambda x: x['results']['Weighted Average Trade'], reverse=True)

In [ ]:
def verifyStrategy(strategy):
    try:
        ticket = strategy['ticket']
        history = getData(ticket, period=1000,
                          interval=strategy["strategy"]["timeFrame"])
        if strategy['strategy']['name'] == 'SmaCrossStrategy':
            smaCurta = int(strategy['strategy']['params'][0])
            smaLonga = int(strategy['strategy']['params'][1])
            result = SmaCrossSignal(
                history, smaCurta=smaCurta, smaLonga=smaLonga)
            return result
        elif strategy['strategy']['name'] == 'HiLoStrategy':
            smaHigh = int(strategy['strategy']['params'][0])
            smaLow = int(strategy['strategy']['params'][1])
            result = HiLoSignal(
                history, smaHigh=smaHigh, smaLow=smaLow)
            return result
    except AttributeError:
        return None
    except Exception as e:
        print("Erro na verificação da estratégia. Erro:")
        print(e)
        return None


In [ ]:
dbcompras = TinyDB(os.path.join('..', 'DB', 'compras.json'))
tabelaOrdens = dbcompras.table('ordens')
ListaOrdens = tabelaOrdens.all()
tabelaDeComprados = dbcompras.table('comprados')
listaDeComprados = tabelaDeComprados.all()

In [ ]:
def extraCondition(estrategia):
    estrategy = estrategia['strategy']
    valor =  estrategy['name'] == 'HiLoStrategy' and 1 in estrategy['params']
    valor = not valor
    return  valor

In [ ]:
listaDeVendas = []
listaDeAtivosVenda = []
for estrategia in listaDeComprados:
    result = verifyStrategy(estrategia)
    if result is not None:
        result.pop('df')
        if result['signal'].lower() == 'sell' and estrategia['ticket'] not in listaDeAtivosVenda:
            estrategia['signal'] = {"signal": 'sell', "close": result['close']}
            estrategia['OrderDate'] = dt.datetime.now().strftime(
                "%Y-%m-%d %H:%M:%S")
            print("Removido: ", estrategia['ticket'])
            listaDeVendas.append(estrategia)
            listaDeAtivosVenda.append(estrategia['ticket'])
            tabelaDeComprados.remove(doc_ids=[estrategia.doc_id])

tabelaOrdens.insert_multiple(listaDeVendas)
ListaOrdens = tabelaOrdens.all()
listaDeComprados = tabelaDeComprados.all()


In [ ]:
sell = tabelaOrdens.search(lambda x: x['signal']['signal'] == 'sell')
len(sell)

In [ ]:
# remove listaDeVendas from tabelaDeComprados
# condicao = lambda x: x['ticket']  in listaDeAtivosVenda['ticket'] and x['ticket'] == listaDeAtivosVenda['ticket']
for venda in sell:
    for comprado in tabelaDeComprados.all():
        if comprado['ticket'] == venda['ticket'] and comprado["strategy"]["name"] == venda["strategy"]["name"] and comprado["strategy"]["params"] == venda["strategy"]["params"] and comprado["strategy"]["timeFrame"] == venda["strategy"]["timeFrame"]:
            print(comprado['ticket'], " ", venda['ticket'])
            print(comprado['strategy'], " ", venda['strategy'])
            tabelaDeComprados.remove(doc_ids=[comprado.doc_id])    


In [12]:
listaDeAtivos = []
novaListaDeComprados = []
for comprado in listaDeComprados:
    if comprado['ticket'] not in listaDeAtivos:
        listaDeAtivos.append(comprado['ticket'])
        novaListaDeComprados.append(comprado)
for vendido in listaDeVendas:
    if vendido['ticket'] not in listaDeAtivos:
        listaDeAtivos.append(vendido['ticket'])
for estrategia in estrategias:
    if len(novaListaDeComprados) >= 10:
        break
    result = verifyStrategy(estrategia)
    if result is not None:
        if result['signal'] == 'Buy' and estrategia['ticket'] not in listaDeAtivos and extraCondition(estrategia):
            if 'signal' in estrategia.keys():
                estrategia['signal']['buy'] = result['close']
            else:
                result.pop('df')
                estrategia['signal'] = result
                resultado = estrategia['results']
                resultado.pop('Start')
                resultado.pop('End')
                resultado.pop('Volatility (Ann.) [%]')
                resultado.pop('_trades')
                estrategia.pop('updateDate')
                estrategia['OrderDate'] = dt.datetime.now().strftime(
                    "%Y-%m-%d %H:%M:%S")
            listaDeAtivos.append(estrategia['ticket'])
            novaListaDeComprados.append(dict(estrategia))
            tabelaDeComprados.insert(dict(estrategia))
            tabelaOrdens.insert(dict(estrategia))
            print("Adicionado: ", estrategia['ticket'], "total de ativos: ", len(listaDeAtivos))
            print("Adicionado: ", estrategia['ticket'], "total de comprados: ", len(novaListaDeComprados))



Adicionado:  BNX total de ativos:  10
Adicionado:  BNX total de comprados:  8


In [13]:
barra = "    "
resultados = listaDeVendas + novaListaDeComprados
for resultado in resultados:
    if resultado['signal']['signal'].lower() == 'sell':
        print('Venda: ', resultado['ticket'].replace('USDT', ''))
    if resultado['signal']['signal'].lower() == 'buy':
        print('Compra: ', resultado['ticket'].replace('USDT', ''))
    print(barra, 'valor: ',  '$', resultado['signal']['close'])
    print(barra, 'chance de acerto: ',  int(
        resultado['results']['Win Rate [%]']), '%')
    print(barra, 'Total de Trades: ',  int(resultado['results']['# Trades']))
    print(barra, 'Média de perda quando erra: ',  int(
        resultado['results']['Avg. Drawdown [%]']), '%')
    print(barra, 'Média de lucro quando acerta: ',  int(
        resultado['results']['Avg. Trade [%]']*resultado['results']['Profit Factor']), '%')
    print(barra, 'Fator de Lucro: ',  int(
        resultado['results']['Profit Factor']))
    print(barra, 'Média de Lucro de Todos os Trades: ',
          int(resultado['results']['Avg. Trade [%]']), '%')


Venda:  POWR
     valor:  $ 0.2618
     chance de acerto:  81 %
     Total de Trades:  94
     Média de perda quando erra:  -32 %
     Média de lucro quando acerta:  9814 %
     Fator de Lucro:  77
     Média de Lucro de Todos os Trades:  126 %
Venda:  TRX
     valor:  $ 0.0724
     chance de acerto:  49 %
     Total de Trades:  63
     Média de perda quando erra:  -20 %
     Média de lucro quando acerta:  266 %
     Fator de Lucro:  7
     Média de Lucro de Todos os Trades:  34 %
Compra:  BTC
     valor:  $ 46827.76
     chance de acerto:  74 %
     Total de Trades:  167
     Média de perda quando erra:  -16 %
     Média de lucro quando acerta:  18742 %
     Fator de Lucro:  91
     Média de Lucro de Todos os Trades:  205 %
Compra:  SYS
     valor:  $ 0.6878
     chance de acerto:  85 %
     Total de Trades:  77
     Média de perda quando erra:  -29 %
     Média de lucro quando acerta:  46698 %
     Fator de Lucro:  190
     Média de Lucro de Todos os Trades:  244 %
Compra:  GNO
     